# Variational eigensolver for toy Hamiltonian on a single qubit
Jordan Fox 3/2/20

This notebook uses the variational technique to solve for the ground state of $H = a Z + b X + c Y $
where $Z,X,Y$ are Pauli matrices.

$a,b,c$ are real inputs.

I use a variational form $R_y (\phi) R_z (\theta) |0\rangle$ and the COBYLA optimizer. If you change `n_params` to 1 it will instead use $R_y (\phi) |0\rangle$. The ground state is printed in this form at the end of optimization.

I have also manually normalized for conservation of probability, even though non-conservation of probability is only due to statistical fluctuations.

Note that computing expectation values of a Pauli operator requires first applying the gate(s) that diagonalize that Pauli (i.e. $H$ for $X$ and $ZSH$ for $Y$).

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.aqua.components.optimizers import COBYLA

from qiskit.visualization import plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000
verbose = True

In [ ]:
def exp_from_counts(counts):
    if '0' not in counts.keys():
        exp_val = -1* counts['1'] / NUM_SHOTS 
    elif '1' not in counts.keys():
        exp_val = counts['0'] / NUM_SHOTS 
    else:
        exp_val = (counts['0'] - counts['1']) / NUM_SHOTS 
    return exp_val

def get_var_form(params,meas_axis='z'):
    # variational form = Ry(theta)Rz(phi)
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.ry(params[0], qr[0])
    if len(params)>1:
        qc.rz(params[1], qr[0])
    if meas_axis == 'x':
        qc.h(0)
    elif meas_axis == 'y':
        qc.z(0)
        qc.s(0)
        qc.h(0)
    qc.measure(qr, cr[0])
    return qc

def ham_exp_val(params,ham_coef):
    # params is a list of variational form parameters i.e. [theta,phi]
    # ham_coef a list [a,b,c] such that the hamiltonian is H = aZ + bX + cY
    qc_z = get_var_form(params,'z')
    result_z = execute(qc_z, backend, shots=NUM_SHOTS).result()
    counts_z = result_z.get_counts()
    
    qc_x = get_var_form(params,'x') 
    result_x = execute(qc_x, backend, shots=NUM_SHOTS).result()
    counts_x = result_x.get_counts()
    
    qc_y = get_var_form(params,'y') 
    result_y = execute(qc_y, backend, shots=NUM_SHOTS).result()
    counts_y = result_y.get_counts()
    
    
    exp_val_vec = np.array([exp_from_counts(counts_z) , exp_from_counts(counts_x), exp_from_counts(counts_y)])
    exp_val_vec = exp_val_vec/np.linalg.norm(exp_val_vec)     #manually normalize probability
    ham_vec = np.multiply(ham_coef,exp_val_vec)
    
    exp_val = sum(ham_vec) 
    
    if verbose:
        print("<Z> = %2.3f , <X> = %2.3f , <Y> = %2.3f " % tuple(exp_val_vec))
        print("Total Probability = %2.3f" % np.linalg.norm(exp_val_vec))
    return exp_val


In [ ]:
ham_coef = np.array([1.,2.,0.5])   #these are the Hamiltonian coefficients on Z, X, and Y respectively
n_params = 2


print('H = %2.3f * Z   +  %2.3f * X +  %2.3f * Y' % tuple(ham_coef))

def objective_function(params):
    return ham_exp_val(params,ham_coef)

optimizer = COBYLA(maxiter=500, tol=10**-8)

params_init = np.random.rand(n_params)
# params_init = np.zeros(n_params)
opt_result = optimizer.optimize(num_vars=n_params, objective_function=objective_function, initial_point=params_init)

exp_ham = ham_exp_val(opt_result[0],ham_coef)
print('< H > computed =  %2.3f' % exp_ham)

ham_matrix = np.array([ [ham_coef[0], ham_coef[1] - 1.j*ham_coef[2]] , [ham_coef[1]+ 1.j*ham_coef[2], -1.*ham_coef[0]]   ])
print('< H > exact = %2.3f' % min(np.linalg.eig(ham_matrix)[0]))
if n_params==1:
    phi = opt_result[0]
    if verbose: print('|g.s.> = Ry(%2.3f) |0>' % phi)
#     print('<z> = %f' % np.cos(phi))
elif n_params==2:
    theta, phi = tuple(opt_result[0])
    if verbose: print('|g.s.> = Rz(%2.3f) Ry(%2.3f) |0>' % (theta,phi))
#     print('<z> = %f' % np.cos(phi))
#     print('<x> = %f' % (2.*np.cos(theta)*np.cos(phi/2.)*np.sin(phi/2.)))